In [1]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup

In [2]:
england = requests.get('http://www.livescores.com/soccer/england/premier-league/')
spain = requests.get('http://www.livescores.com/soccer/spain/primera-division/')
france = requests.get('http://www.livescores.com/soccer/france/ligue-1/')
italy = requests.get('http://www.livescores.com/soccer/italy/serie-a/')
germany = requests.get('http://www.livescores.com/soccer/germany/bundesliga/')

In [73]:
def get_team_stats(country):
    soup = BeautifulSoup(country.text, 'lxml')
    league_soup = soup.find("div", {"class":"ltable table"})
    
    teams = league_soup.find_all("div", {"class":"team"})
    stats = league_soup.find_all("div", {"class":"pts"})
    
    return {'teams': teams, 'stats': stats}


def get_ordered_stats(country):
    unordered_stats = get_team_stats(country)
    
    teams = unordered_stats['teams']
    stats = unordered_stats['stats']
    
    team_names = []
    all_stats_ungrouped = []
    all_stats_grouped = []

    [team_names.append(team.text) for team in teams]
    [all_stats_ungrouped.append(stat.text) for stat in stats]
    [all_stats_grouped.append(all_stats_ungrouped[i : i + 8]) for i in range(0, len(all_stats_ungrouped), 8)]

    columns = [team_names.pop(0)] + all_stats_grouped.pop(0)

    [all_stats_grouped[i].insert(0, team_names[i]) for i in range(len(all_stats_grouped))]

    table = pd.DataFrame(all_stats_grouped, columns=columns, index=range(1, len(all_stats_grouped) + 1))
    convert_dict = {'P':int,'W':int,'D':int, 'L':int, 'F':int,'A':int,'GD':int, 'Pts':int}
    table = table.astype(convert_dict)
    return table
    
epl = get_ordered_stats(england)
la_liga = get_ordered_stats(spain)
ligue_one = get_ordered_stats(france)
bundesliga = get_ordered_stats(germany)
serie_a = get_ordered_stats(italy)

In [92]:
combined_table = pd.concat([epl, la_liga, ligue_one, bundesliga, serie_a], axis=0)
combined_table = combined_table.sort_values(by=['Pts', 'GD', 'P'], ascending=[False, False, True])
combined_table.index = range(1, len(combined_table) + 1)
combined_table

,Team Name,P,W,D,L,F,A,GD,Pts
1,Liverpool,29,27,1,1,66,21,45,82
2,Paris Saint-Germain,27,22,2,3,75,24,51,68
3,Juventus,26,20,3,3,50,24,26,63
4,Lazio,26,19,5,2,60,23,37,62
5,Barcelona,27,18,4,5,63,31,32,58
...,...,...,...,...,...,...,...,...,...
94,SPAL,26,5,3,18,20,44,-24,18
95,Werder Bremen,24,4,6,14,27,55,-28,18
96,SC Paderborn 07,25,4,4,17,30,54,-24,16
97,Brescia,26,4,4,18,22,49,-27,16
